In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import os
import pandas as pd
from os.path import exists
from pathlib import Path
import logging
COBRAKBase_path = "/Users/chenry/code/cobrakbase"
BaseModule_path = "/Users/chenry/code/KBBaseModules"
ModelSEEDpy_path = "/Users/chenry/code/ModelSEEDpy"
ModelSEEDReconstruction_path = "/Users/chenry/code/KB-ModelSEEDReconstruction"
sys.path.append(COBRAKBase_path)
sys.path.append(BaseModule_path)
sys.path.append(ModelSEEDpy_path)
sys.path.append(ModelSEEDReconstruction_path+"/lib")
from modelseedpy.core.mstemplate import MSTemplateBuilder
from ModelSEEDReconstruction.modelseedrecon import ModelSEEDRecon
from installed_clients.WorkspaceClient import Workspace
#You need to get a KBase dev token or pull a token from the narrative and put in ~/.kbase/token file
token = None
if Path(str(Path.home()) + '/.kbase/token').exists():
    with open(str(Path.home()) + '/.kbase/token', 'r') as fh:
        token = fh.read().strip()
#workspace_url = "https://appdev.kbase.us/services/ws"#For appdev
workspace_url = "https://kbase.us/services/ws"#For prod

python version 3.7.6
cobrakbase 0.2.8


In [2]:
wsclient = Workspace(workspace_url, token=token)
msrecon = ModelSEEDRecon({"ATP_media_workspace":"68393","version":"0.0.1"},ModelSEEDReconstruction_path,ModelSEEDReconstruction_path+"/notebooks/",token,{"Workspace":wsclient},None)

In [3]:
template = msrecon.kbase_api.get_object("GramNegModelTemplateV4","NewKBaseModelTemplates")
template = MSTemplateBuilder.from_dict(template).build()
biomass = template.biomasses.get_by_id("bio1")

In [4]:
bsub168 = msrecon.kbase_api.get_from_ws("bsub168.mdl",135225)
ecoliK12 = msrecon.kbase_api.get_from_ws("ecoliK12.mdl",135225)
ecoli_bio_MS1 = ecoliK12.reactions.get_by_id("bio1")
ecoli_bio_1_new = biomass.build_biomass(ecoliK12, index="0",classic=False,GC=0.508)
ecoli_bio_1_classic = biomass.build_biomass(ecoliK12, index="0",classic=True,GC=0.508)
data = {}
for met in ecoli_bio_MS1.metabolites:
    data[met.id] = {
        "ms1":ecoli_bio_MS1.metabolites[met],
        "new":0,
        "classic":0
    }
for met in ecoli_bio_1_new.metabolites:
    if met.id not in data:
        data[met.id] = {
            "ms1":0,
            "new":ecoli_bio_1_new.metabolites[met],
            "classic":0
        }
    else:
        data[met.id]["new"] = ecoli_bio_1_new.metabolites[met]
for met in ecoli_bio_1_classic.metabolites:
    if met.id not in data:
        data[met.id] = {
            "ms1":0,
            "new":0,
            "classic":ecoli_bio_1_classic.metabolites[met]
        }
    else:
        data[met.id]["classic"] = ecoli_bio_1_classic.metabolites[met]
df = pd.DataFrame(data)
df = df.transpose()
df.to_csv("BiomassComparison.csv")

In [5]:
rxn = ecoliK12.reactions.get_by_id("rxn05295_c0")
rxn

KeyError: 'rxn05295_c0'

In [4]:
template.overwrite_biomass_from_table("biomasses.tsv","bio1",biomass.name,biomass.type,biomass.dna,biomass.rna,biomass.protein,biomass.lipid,biomass.cellwall,biomass.cofactor,biomass.energy,biomass.other)
newbiomass = template.biomasses.get_by_id("bio1")

In [ ]:
output = msrecon.build_metabolic_models({
    "workspace":68304,
    "genome_refs":["68303/Escherichia_coli_K-12_MG1655.RAST"],
    "run_gapfilling":True,
    "atp_safe":True,
    "forced_atp_list":[],
    "gapfilling_media_list":["KBaseMedia/Carbon-D-Glucose"],
    "suffix":".mdl",
    "core_template":"auto",
    "gs_template":"auto",
    "gs_template_ref":None,
    "template_reactions_only":True,
    "output_core_models":True
})

{
    "workspace": 68304,
    "genome_refs": [
        "68303/Escherichia_coli_K-12_MG1655.RAST"
    ],
    "run_gapfilling": true,
    "atp_safe": true,
    "forced_atp_list": [],
    "gapfilling_media_list": [
        "KBaseMedia/Carbon-D-Glucose"
    ],
    "suffix": ".mdl",
    "core_template": "auto",
    "gs_template": "auto",
    "gs_template_ref": null,
    "template_reactions_only": true,
    "output_core_models": true
}


/Users/chenry/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318 UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
